# A full example
> A more detailed example on how we use torchember

### How to read this notebook in 30 sec
* Even though this is a tiny walk through version, most of the code has nothing to do with torchember
* The lines of codes that's relate to the ```torchember``` are marked with 
```python
# ========= TORCHEMBER CODE ===========
```

### Scope of this notebook
* Most of the codes in this notebook has nothing to do with torchember
* The training framework is optional, you can use fast.ai, catalyst or forgebox or you just write the iteration all by yourself

### Possible extra dependency beside anaconda
* pytorch: ... that goes without saying
* torchvision: ```pip install torchvision```
* forgebox: ```pip install forgebox```
* torchember: ```pip install torchember```

In [1]:
import torch
from torch import nn
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as tfm
from forgebox.ftorch.train import Trainer
from forgebox.ftorch.prepro import test_DS

In [2]:
from pathlib import Path
import os
import json
import numpy as np
import pandas as pd

### Create Pytorch Dataset

Image trainsformtion funcitons

In [3]:
transforms = tfm.Compose([
    tfm.ToTensor(),
    tfm.Normalize(mean = [.5,],std = [.5,]),
])

In [4]:
HOME  = Path(os.environ['HOME'])
DATA = HOME/"data"
DATA.mkdir(exist_ok = True)

In [5]:
train_path = DATA/"mnist.train"
valid_path = DATA/"mnist.valid"

In [6]:
train_set = MNIST(train_path,train=True, 
                  download=False if (train_path/"MNIST").exists() else True,
                  transform=transforms)
valid_set = MNIST(valid_path,train=False, 
                  download=False if (valid_path/"MNIST").exists() else True,
                  transform=transforms)

Test the shape of example x,y, tensor

In [7]:
x,y = test_DS(train_set)()
x.shape, y.shape

(torch.Size([1, 1, 28, 28]), torch.Size([1]))

### Pytorch Model

In [8]:
def conv2d(in_, out_,ks=3,bias=True):
    return nn.Conv2d(in_,out_, kernel_size=ks, padding=ks//2,bias = bias)

class vggBlock(nn.Module):
    def __init__(self, in_,out_,nb_layers):
        super().__init__()
        self.in_ = in_
        self.out_ = out_
        self.nb_layers = nb_layers
        self.seq = nn.Sequential(*self.create_convs())
        
    def forward(self,x):
        return self.seq(x)
        
    def create_convs(self,):
        layers = []
        for l in range(self.nb_layers):
            if l == 0 : layers.append(conv2d(self.in_,self.out_))
            else:layers.append(conv2d(self.out_,self.out_))
                
            layers.append(nn.BatchNorm2d(self.out_))
            layers.append(nn.LeakyReLU())
        layers.append(nn.MaxPool2d(2))
        return layers

class tinyVGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(*[
            vggBlock(1,16,2),
            vggBlock(16,32,2),
            vggBlock(32,64,2),
        ])
        self.fcb = nn.Sequential(*[
            nn.Linear(64*3*3, 256,bias = True),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,10),
            nn.BatchNorm1d(10),
        ])
        
    def forward(self,x):
        bs = x.size(0)
        x = self.features(x).view(bs,-1)
        x = self.fcb(x)
        return x
        

### Arm the model

In [9]:
model =tinyVGG()
from torchember.core import torchEmber # ========= TORCHEMBER CODE ===========
te = torchEmber(model) # ========= TORCHEMBER CODE ===========

start analyzing model


In [10]:
train = Trainer(train_set,val_dataset=valid_set,)

Notice, The Trainer was not initiated with optimizer
            Use the following syntax to initialize optimizer
            t.opt["adm1"] = torch.optim.Adam(m1.parameters())
            t.opt["adg1"] = torch.optim.Adagrad(m2.parameters())
            


Define optimizers

In [11]:
train.opt["adm"] = torch.optim.Adam(model.parameters())

Define loss function

In [12]:
loss_func = nn.CrossEntropyLoss()

Metric function

In [13]:
def guess(x):
    return torch.max(x,-1).indices
def accuracy(y_,y):
    return (guess(y_)==y).float().mean()

Define training and validation step

In [14]:
@train.step_train
def action(batch):
    if batch.i ==0:
        te.mark(phase = "train") # ========= TORCHEMBER CODE =========== (optional)
        te.mark(epoch = batch.epoch) # ========= TORCHEMBER CODE =========== (optional)
    batch.opt.zero_all()
    x,y = batch.data
    y_ = model(x)
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    loss.backward()
    batch.opt.step_all()
    te.log_model()  # ========= TORCHEMBER CODE =========== (optional)
    return {"loss":loss.item(),"acc":acc.item()}

@train.step_val
def val_action(batch):
    if batch.i ==0:
        te.mark(phase = "valid") # ========= TORCHEMBER CODE =========== (optional)
        te.mark(epoch = batch.epoch) # ========= TORCHEMBER CODE =========== (optional)
    x,y = batch.data
    y_ = model(x)
    loss = loss_func(y_)
    acc = accuracy(y_,y)
    return {"loss":loss.item(),"acc":acc.item()}


In [ ]:
train.train(2)